# **Import Required Libraries**

In [ ]:
from bs4 import BeautifulSoup
#from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import json
import os
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from datetime import datetime, timedelta

In [ ]:
url = "https://www.facebook.com/Hespress/videos"
number_of_scroll=20

# **Function to Load Videos: Titles, Likes, Views, Links**

In [ ]:
def get_post_vid(url, nbr_scroll):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument("--disable-notifications")

  driver = webdriver.Chrome(executable_path=r'',options=chrome_options)


  driver.get(url)

  time.sleep(5)

  email_input = driver.find_element_by_name("email")
  password_input = driver.find_element_by_name("pass")

  # Enter your email and password
  email_input.send_keys("")
  password_input.send_keys("")

  # Find and click the "Se connecter" button
  se_connecter_button = driver.find_element(By.XPATH, f'//div[contains(@class, "x1i10hfl x1qjc9v5 xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli x1n2onr6 x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x3nfvp2 x1q0g3np x87ps6o x1lku1pv x1a2a7pz xtvsq51 xhk9q7s x1otrzb0 x1i1ezom x1o6z2jb x1vqgdyp x6ikm8r x10wlt62 xexx8yu xn6708d x1120s5i x1ye3gou")]')  # Replace with the actual name attribute of the button
  driver.execute_script("arguments[0].click();", se_connecter_button)


  for _ in range(nbr_scroll):
      driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
      time.sleep(2)

  html = driver.page_source
  soup = BeautifulSoup(html, "html.parser")

  container_div = soup.find("div", class_="x1qjc9v5 x1lq5wgf xgqcy7u x30kzoy x9jhf4c x78zum5 xdt5ytf x1l90r2v xyamay9 xjl7jj")
  data_table = []
  try:
      if container_div:

          pubs = container_div.find_all("div", class_="x9f619 x1r8uery x1iyjqo2 x6ikm8r x10wlt62 x1n2onr6")

          for pub in pubs:

              title = pub.find("span", class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
              title = title.text.strip().replace('\xa0', ' ').replace('\n', ' ') if title else "Title not found"

              views = pub.find("span", class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa xo1l8bm x12scifz x1yc453h").find_next("div")
              views = views.text.strip() if views else "0"
              views = views.replace('\xa0', '')
              if re.search(r'\bvue\w*\b', views):
                  pass
              else:
                  views = "0 vue"

              like = pub.find("span",class_="xt0b8zv xjbqb8w xrbpyxo x1mh8g0r x6ikm8r x10wlt62 xlyipyv xuxw1ft")
              like = like.text.strip().replace('\xa0', '') if like else "0"

              link = pub.find("a",class_="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv")
              link = link.get('href').strip() if link else "link not found"

              data_table.append({"Title":title,"Like":like,"Views":views,"Link":link})

  except AttributeError as e:
      print(f"Error: {e}")
      views = "Views not found"

  driver.quit()

  return data_table

# **Function to Load Number of Shares and Date**

In [ ]:
def get_share(url):

  share_table= []
  mobile_emulation = {
      "deviceMetrics": {"width": 360, "height": 640, "pixelRatio": 3.0},
      "userAgent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36",
  }

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)

  driver = webdriver.Chrome(executable_path=r'C:\Users\khali\Desktop\chromedriver-win64\chromedriver.exe', options=chrome_options)

  url =driver.get(url)
  time.sleep(4)

  se_connecter_button = driver.find_element(By.XPATH, f'//div[contains(text(), "󱤅")]')
  driver.execute_script("arguments[0].click();", se_connecter_button)
  time.sleep(5)
  for _ in range(2):
      driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
      time.sleep(2)

  driver.implicitly_wait(10)

  # Get the updated page source
  page_source = driver.page_source

  # Parse the page source with BeautifulSoup
  soup = BeautifulSoup(page_source, 'html.parser')

  # Find the button with class 'native-text'
  button_element = soup.find_all('button', class_='native-text')
  if (len(button_element[3].text.split(' '))>=2):
      share_table.append(button_element[3].text.split(' ')[1])
  else:
      share_table.append("0")

  element = soup.find('div',class_='m')

  if element:
      week = element.find_all('span',class_="f5")
      week = week[0].text.replace("\U000f078b\U000f07e1","").replace("\U000f078b","")
      share_table.append(week)
  driver.quit()
  return share_table

# **Function for Normalizing Dates**

In [ ]:
def update_date(date_str):

    current_year = "2023"
    date_str = date_str.strip()
    current_year = date_str.split()[-1] if date_str.count(' ') > 1 else None
    now = datetime.now()

    if 'h' in date_str:
        specified_hour = int(date_str.split('h')[0])
        hour_difference = now.hour - specified_hour
        if hour_difference < 0:
            return (now - timedelta(days=1)).strftime("%d/%m/%Y")
        else:
            return now.strftime("%d/%m/%Y")
    elif ('j' in date_str) and (len(date_str) == 3):
        days_ago = int(date_str.split()[0])
        updated_date = now - timedelta(days=days_ago)
        return updated_date.strftime("%d/%m/%Y")
    elif 'min' in date_str:
        return now.strftime("%d/%m/%Y")
    else:
        parts = date_str.split()
        day = int(parts[0])
        month_name = parts[1]
        # Map month names to month numbers
        month_dict = {
            'janv.': 1, 'févr.': 2, 'mars': 3, 'avr.': 4, 'mai': 5, 'juin': 6,
            'juil.': 7, 'août': 8, 'sept.': 9, 'oct.': 10, 'nov.': 11, 'déc.': 12
        }

        month = month_dict.get(month_name, 1)

        # Add the year if specified
        if current_year and current_year.isdigit():
            year = int(current_year)
        else:
            year = now.year

        updated_date = datetime(year, month, day)
        return updated_date.strftime("%d/%m/%Y")

# **Function to Retrieve Comments and Duration**

In [ ]:
def get_comment(url):
    comments_list=[]

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-notifications")

    driver = webdriver.Chrome(executable_path=r'C:\Users\khali\Desktop\chromedriver-win64\chromedriver.exe',options=chrome_options)
    driver.set_window_size(1300, 800)
    driver.get(url)

    time.sleep(5)

    email_input = driver.find_element_by_name("email")
    password_input = driver.find_element_by_name("pass")


    email_input.send_keys("njr36965@gmail.com")
    password_input.send_keys("wydadac")

    se_connecter_button = driver.find_element(By.XPATH, f'//div[contains(@class,"x1i10hfl x1qjc9v5 xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli x1n2onr6 x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x3nfvp2 x1q0g3np x87ps6o x1lku1pv x1a2a7pz xtvsq51 xhk9q7s x1otrzb0 x1i1ezom x1o6z2jb x1vqgdyp x6ikm8r x10wlt62 xexx8yu xn6708d x1120s5i x1ye3gou")]')
    driver.execute_script("arguments[0].click();", se_connecter_button)

    time.sleep(8)

    tous_comments = driver.find_element(By.XPATH,f'//span[contains(@class, "x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen x1s688f xi81zsa")]')
    driver.execute_script("arguments[0].click();", tous_comments)

    time.sleep(5)
    tous_comments_1 = driver.find_element(By.XPATH,f'//span[contains(text(), "Tous les commentaires")]')
    driver.execute_script("arguments[0].click();", tous_comments_1)

    time.sleep(5)

    element_to_find = ['Voir', 'Afficher']
    span_element=None
    for text in element_to_find:
        try:
            span_element = driver.find_element(By.XPATH, f'//span[contains(text(), "{text}")]')
            break

        except NoSuchElementException:
            pass

    if span_element is not None:
        for i in range(10):
            try:
                driver.execute_script("arguments[0].click();", span_element)
                time.sleep(5)
            except:
                break

    time.sleep(10)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")


    comment = soup.find("span",class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
    comment = comment.text.replace('\xa0', '') if comment else "0"
    if re.search(r'\bcomment\w*\b', comment):
        pass
    else:
        comment = "0 commentaire"
    comments_list.append(comment)

    dure = soup.find_all("div",class_="x14ctfv x1rg5ohu x1pg5gke xss6m8b x7h9g57 x1t4t16n x8j4wrb x9hgts1 x2b8uid x27saw0 x3ajldb")
    dure = (dure[0].text).split('/')[1] if dure else "Dure not found"
    comments_list.append(dure)

    container_div = soup.find("div", class_="x78zum5 xdt5ytf x6ikm8r x1odjw0f x1iyjqo2 x1pi30zi x1swvt13")
    if container_div:
        pubs = container_div.find_all("div", class_="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs")
        list_comment = []
        for pub in pubs:
            list_comment.append(pub.text.replace('\xa0', ' ').replace('\n', ' '))
        comments_list.append(list_comment)

    driver.quit()

    return comments_list

## **Compare Date**

In [ ]:
def compare_date(post_date, mydate)

  date_p = datetime.strptime(post_date, "%d/%m/%Y")
  date = datetime.strptime(mydate, "%d/%m/%Y")

  if date_p > date:
      return True
  else :
      return False

## **Retrieve video posts and check whether they fall within the desired date range.**

In [ ]:
url = "  "
date = "14/07/2023"
number_scroll = 5

video_post = get_post_vid(url, number_scroll)
last_video = video_post[-1]["Link"]

date_last_video = update_date(get_share(last_video)[1])
ok = compare_date(date_last_video,date)

while (ok==False):
  number_scroll += 5
  video_post = get_post_vid(url, number_scroll)
  last_video = video_post[-1]["Link"]

  date_last_video = update_date(get_share(last_video)[1])
  ok = compare_date(date_last_video,date)

NameError: ignored